In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark= SparkSession.builder.getOrCreate()

In [3]:
matches=spark.read.format('csv') \
    .option('header','True') \
    .option('inferschema','True') \
    .load('../data/matches.csv')

In [4]:
matches.show(3)

+-------+----+----------+-------------------+----------+----+------------+-------------+----------------+----------+-------------+--------------+
| Season|Tour|      Date|               Time|  Opponent|HoAw|ArsenalScore|OpponentScore|         Stadium|Attendance|        Coach|       Referee|
+-------+----+----------+-------------------+----------+----+------------+-------------+----------------+----------+-------------+--------------+
|2017/18|   1|2017-08-11|2025-03-22 20:45:00| Leicester|home|           4|            3|Emirates Stadium|     59387|Arsène Wenger|     Mike Dean|
|2017/18|   2|2017-08-19|2025-03-22 18:30:00|Stoke City|away|           0|            1|  bet365 Stadium|     29459|Arsène Wenger|Andre Marriner|
|2017/18|   3|2017-08-27|2025-03-22 17:00:00| Liverpool|away|           0|            4|         Anfield|     53206|Arsène Wenger|  Craig Pawson|
+-------+----+----------+-------------------+----------+----+------------+-------------+----------------+----------+--------

In [5]:
matches.schema

StructType([StructField('Season', StringType(), True), StructField('Tour', IntegerType(), True), StructField('Date', DateType(), True), StructField('Time', TimestampType(), True), StructField('Opponent', StringType(), True), StructField('HoAw', StringType(), True), StructField('ArsenalScore', IntegerType(), True), StructField('OpponentScore', IntegerType(), True), StructField('Stadium', StringType(), True), StructField('Attendance', IntegerType(), True), StructField('Coach', StringType(), True), StructField('Referee', StringType(), True)])

In [6]:
matches.createOrReplaceTempView("Matches")

In [7]:
distinct_matches=spark.sql('''
    select count(distinct Date) 
    from Matches

''').show()

+--------------------+
|count(DISTINCT Date)|
+--------------------+
|                 214|
+--------------------+



In [8]:
Matches=spark.sql("""
    select count(Date)
    from Matches
""").show()

+-----------+
|count(Date)|
+-----------+
|        214|
+-----------+



In [9]:
matches.columns

['Season',
 'Tour',
 'Date',
 'Time',
 'Opponent',
 'HoAw',
 'ArsenalScore',
 'OpponentScore',
 'Stadium',
 'Attendance',
 'Coach',
 'Referee']

In [10]:
DimMatch= matches.withColumn("MatchID", monotonically_increasing_id())

In [11]:
DimMatch.columns

['Season',
 'Tour',
 'Date',
 'Time',
 'Opponent',
 'HoAw',
 'ArsenalScore',
 'OpponentScore',
 'Stadium',
 'Attendance',
 'Coach',
 'Referee',
 'MatchID']

In [13]:
DimMatch.write.csv('../data/DimMatches', header=True)